In [61]:
from bs4 import BeautifulSoup
from datetime import datetime
import re
import polars as pl

In [62]:
def extract_info(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    results = []

    for item in soup.find_all('div', class_='view-content'):
        for store in item.find_all('div', recursive=False):
            name_div = store.find('div', class_='views-field views-field-title')
            if name_div:
                name = name_div.find('a').text.strip()
                
                # Search for date in the entire store div
                date_text = store.find(string=re.compile(r'\d{2}/\d{2}/\d{4}'))
                if date_text:
                    date_str = date_text.strip()
                    try:
                        date = datetime.strptime(date_str, '%d/%m/%Y')
                        trimester = (date.month - 1) // 3 + 1
                        results.append((name, f"{trimester}T{date.year}"))
                    except ValueError:
                        results.append((name, "Invalid date format"))
                else:
                    results.append((name, "Date not found"))
            
    return results

def process_html(html_content):
    info = extract_info(html_content)
    df = pl.DataFrame(info)
    # for name, trimester in info:
        # print(f"Name: {name}")
        # print(f"Trimester: {trimester}")
        # print("-" * 30)
    return df


In [63]:
# Define the path to your HTML file
file_path = 'data_2022.html'

# Read the HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

results = process_html(html_content)
results = results.transpose()
results.write_csv('T2022.csv')

In [64]:
# Define the path to your HTML file
file_path = 'data_2023.html'

# Read the HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

results = process_html(html_content)
results = results.transpose()
results.write_csv('T2023.csv')

In [65]:
# Define the path to your HTML file
file_path = 'data_2024.html'

# Read the HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

results = process_html(html_content)
results = results.transpose()
results.write_csv('T2024.csv')